In [1]:
import zmq
import cv2
import os
import numpy as np
import PyUrComu.UrCommunication as UrCommunication
from scipy.spatial.transform import Rotation as R
from scipy.spatial import distance
import time
from colorama import Fore, Back, Style
import pickle as pkl
from matplotlib import pyplot as plt
import open3d as o3d
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition
from skimage.measure import LineModelND, ransac
import traceback
from scipy.spatial.transform import Rotation as R

# global variables and elements

In [2]:
class THT_Element:
    griffpunkt = []
    drahtdicke = 0.0007
    scanpunkt = [] #False-gerade, True-quer
      
    # parameterized constructor
    def __init__(self, griffpunkt, drahtdicke,scanorientierung):
        self.griffpunkt = griffpunkt
        self.drahtdicke = drahtdicke
        if scanorientierung==False:
            self.scanpunkt=[-0.445, -0.28, 0.24423, -3.141,-0.008,3.140]
        else:
            self.scanpunkt=[-0.445, -0.28, 0.24423, -3.141,-0.008,-1.625]


In [3]:

zwischen_punkt=[-0.46538, -0.15051, 0.36951, -3.141,-0.008,3.140]
griffpunkte=[
            [-0.59600, 0.08, 0.27972, -3.141,-0.008,3.140],  ## kondensator erst gerade
   
            [-0.59400, 0.26, 0.270, -3.141,-0.008, 3.140],  ## braun gerade
    
            [-0.38050, 0.26, 0.272, -3.141,-0.008,-1.625], ## grün 30 quer
            [-0.43070, 0.26, 0.272, -3.141,-0.008,-1.625], ## grün 30 quer
            [-0.48030, 0.26, 0.272, -3.141,-0.008,-1.625], ## grün 30 quer
            [-0.53100, 0.26, 0.272, -3.141,-0.008,-1.625], ## grün 30 quer
    
   
    
            [-0.4874, 0.08, 0.27972, --3.141,-0.008,3.140], ## kondensator quer
            [-0.4265, 0.08, 0.27972, -3.141,-0.008,-1.625], ## kondensator quer
    
       
            [-0.5560, 0.08, 0.27972, -3.141,-0.008,3.140]] ## kondensator last gerade
startx=-0.445
scanpunkt=[startx, -0.28, 0.24423, -3.141,-0.008,3.140]#[startx, -0.28, 0.23423, -3.141,-0.008,3.140]
board_anfahr=[-0.21329, -0.43357, 0.279, -3.141,-0.008,3.140]
#scanpunkt_extra=[0.48325, 0.30441, 0.26404,-3.13645596,0.03055905,0.84028594]
start_position_rotation_rpy=[-3.141,-0.008,3.140]
TCP=[0.0000,0.000,0.0000]


In [4]:
# some of the elements extra rotated not parallel to the camera. it s done in order to demonstrate,
# that rotation does not play any role as far as both wires are seen by the camera
tht_elemente=[]
tht_elemente.append(THT_Element(griffpunkte[0],0.0007,False))

tht_elemente.append(THT_Element(griffpunkte[1],0.0007,True))   # not optimal rotation

tht_elemente.append(THT_Element(griffpunkte[2],0.0007,True))
tht_elemente.append(THT_Element(griffpunkte[3],0.0007,True))
tht_elemente.append(THT_Element(griffpunkte[4],0.0007,True))
tht_elemente.append(THT_Element(griffpunkte[5],0.0007,False))  # not optimal rotation

tht_elemente.append(THT_Element(griffpunkte[6],0.0007,False))
tht_elemente.append(THT_Element(griffpunkte[7],0.0007,True))



tht_elemente.append(THT_Element(griffpunkte[8],0.0007,False))

# Functions

In [5]:
def load_transformation_matrix():
    transformation_matrix=None
    if os.path.isfile("transformation_matrix_Photoneo.pkl")==False:
        print("file: ", transformation_matrix," not found")
        return transformation_matrix

    try:
        # load tranformation matrix
        transformation_matrix = pkl.load( open( "transformation_matrix_Photoneo.pkl", "rb" ) )
        print( transformation_matrix )

        # analyze matrix: print rotation values in degrees
        transformation_matrix_analyze=R.from_matrix( transformation_matrix [:3,:3])
        transformation_matrix_analyze=transformation_matrix_analyze.as_euler("xyz")
        print("rotations: ",np.degrees(transformation_matrix_analyze[0]), \
              np.degrees(transformation_matrix_analyze[1]),np.degrees(transformation_matrix_analyze[2]))
        return transformation_matrix
    except Exception as e:
        print(str(e))
        print("error")
        traceback.print_exc()

In [6]:
# taken from https://github.com/nghiaho12/rigid_transform_3D/blob/master/rigid_transform_3D.py
def rigid_transform_3D(A, B):
    assert len(A) == len(B)

    num_rows, num_cols = A.shape;

    if num_rows != 3:
        raise Exception("matrix A is not 3xN, it is {}x{}".format(num_rows, num_cols))

    [num_rows, num_cols] = B.shape;
    if num_rows != 3:
        raise Exception("matrix B is not 3xN, it is {}x{}".format(num_rows, num_cols))

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # subtract mean
    Am = A - np.tile(centroid_A, (1, num_cols))
    Bm = B - np.tile(centroid_B, (1, num_cols))

    H = Am * np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T * U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...\n");
        Vt[2,:] *= -1
        R = Vt.T * U.T

    t = -R*centroid_A + centroid_B

    return R, t


In [7]:
def create_transformation_matrix(picked_points_3d_camera,picked_points_3d_robot): #TODO rewrite this function with NP Arrays
    A=np.asmatrix(picked_points_3d_camera).T
    B=np.asmatrix(np.asanyarray(picked_points_3d_robot)).T
    print("matricies:")
    print(A,B)
    r,t=rigid_transform_3D(A,B)
    print("R",r)
    print("t",t)
    temp_mat_1=np.concatenate((r,t),axis=1)
    temp_mat_2=np.mat((0,0,0,1))
    transformation_matrix=np.array(np.concatenate((temp_mat_1,temp_mat_2),axis=0)) #TODO fix that function to avoid matrixes
    print("Transformation matrix created.")
    print(transformation_matrix)
    # analyze matrix: print rotation values in degrees
    transformation_matrix_analyze=R.from_matrix(transformation_matrix [:3,:3])
    transformation_matrix_analyze=transformation_matrix_analyze.as_euler("xyz")
    print("rotations rX,rY,rZ: ",np.degrees(transformation_matrix_analyze[0]), \
          np.degrees(transformation_matrix_analyze[1]),np.degrees(transformation_matrix_analyze[2]))
    return transformation_matrix

In [8]:
def take_picture(socket):
    socket.send(b"get_wires")
    message = socket.recv()
    #socket.send (b"200")     # one wire
    socket.send (b"10")
    message = socket.recv()
    print ("Received reply from server:", message)

In [9]:
def get_images(socket):
    take_picture(socket)
    wire_1_mat=cv2.imread("images/wire_1.exr", cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    wire_2_mat=cv2.imread("images/wire_2.exr", cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    grayimage=cv2.imread("images/graymat.jpg", 0)
    grayimage = cv2.equalizeHist(grayimage)
    grayimage=cv2.cvtColor(grayimage, cv2.COLOR_GRAY2BGR)
    print("wire_1.exr loaded. Shape: ", wire_1_mat.shape)
    print("wire_2.exr loaded. Shape: ", wire_2_mat.shape)
    return wire_1_mat,wire_2_mat,grayimage

In [10]:
def transform_to_base_coordinate(transformation_matrix,wire_in_cam_coord):
        wire_in_cam_coord_t=np.append(wire_in_cam_coord,1).reshape((-1,1))
        wire_in_cam_coord_rob_coord=np.dot(transformation_matrix[:4,:4],wire_in_cam_coord_t)
        return wire_in_cam_coord_rob_coord.T[0]

In [11]:
def load_and_transform_place_positions(reference_points, scale=0.00001 ):
    """
    reference_points are first and last points in place_positions, but in base system.
    also one more point will be added at the beginning. It has the same coordinate as first real point, 
    but it shifted by z-Axis in base system by +8 mm. It is done for propper transformation.
    The points shoud be estimated by teaching with well defined TCP
    They will be used to transform loaded place positions to robot coordinate system
    
    scale factor between board and robot coordinate system
    """
    place_positions_transformed=[]
    temp_place_positions=[]
    
    if os.path.isfile("place_positions.pkl")==False:
        print("file place_positions not found")
        return place_positions

    try:
        # load points and transform those to 3D form. 
        place_positions_original = pkl.load( open( "place_positions.pkl", "rb" ) )
        
        for position in place_positions_original:
            temp_place_positions.append([(position[0][0]*scale,position[0][1]*scale,0),  # make 3D and
                                         (position[1][0]*scale,position[1][1]*scale,0)]) # transform in meter

        # create coordinate system in board coordinates 
        # for further transformation matrix estimation    
        first_point=temp_place_positions[0][0]
        last_point=temp_place_positions[-1][1]
        board_coordinate_system_points=[(first_point[0],first_point[1],0.08),first_point,last_point]
        
        print("\nboard_coordinate_system_points\n",board_coordinate_system_points)
        print("\nreference_points\n",reference_points)
        board_transformation_matrix=create_transformation_matrix(board_coordinate_system_points,reference_points)

        for temp_point in temp_place_positions:
            temp_point_t_1=np.append(temp_point[0],1).reshape((-1))
            temp_point_t_2=np.append(temp_point[1],1).reshape((-1))
            #print(temp_point_t_1,temp_point_t_2)
            place_positions_transformed.append((np.dot(board_transformation_matrix,temp_point_t_1),
                                               np.dot(board_transformation_matrix,temp_point_t_2)))
        return place_positions_transformed
    except Exception as e:
        print(str(e))
        print("error")
        traceback.print_exc()


In [12]:
def find_wire_end(image_3d,wire_thickness):
    
    # prepare array
    line_3d=np.asanyarray(image_3d).reshape(-1,1,3)
    x_3d=line_3d[:,:,0].reshape(-1)
    y_3d=line_3d[:,:,1].reshape(-1)
    z_3d=line_3d[:,:,2].reshape(-1)
    xyz = np.column_stack((x_3d,y_3d,z_3d))


    # remove zeros
    xyz=xyz[np.where(np.any((xyz!=0),axis=1))]
    xyz_pcd = o3d.geometry.PointCloud()
    xyz_pcd.points = o3d.utility.Vector3dVector(xyz)
    xyz_pcd.paint_uniform_color([0.9, 0.1, 0.1])
    print(len(xyz))

            
    # ransac
    model_robust, inliers = ransac(xyz, LineModelND, min_samples=len(xyz)-1, residual_threshold=0.0001, max_trials=1000)
    outliers = inliers == False
    vector=model_robust.params[1]#-model.params[0]
    print("model.params",model_robust.params)
    print("vector",vector)
    lower_point=model_robust.predict([max(xyz[:,1])][0],axis=1)
        
    # draw wire
    xyz_pcd_inliers = o3d.geometry.PointCloud()
    xyz_pcd_inliers.points = o3d.utility.Vector3dVector(xyz[inliers])
    xyz_pcd_inliers.paint_uniform_color([0.1, 0.9, 0.1])
    #pcd_downsampled = o3d.geometry.PointCloud()
    #pcd_downsampled = pcd.voxel_down_sample(voxel_size=0.002)
    
    # draw lower_point
    lower_point_pcd = o3d.geometry.PointCloud()
    lower_point_pcd.points=o3d.utility.Vector3dVector([lower_point])
    lower_point_pcd.paint_uniform_color([0.1, 0.1, 0.9])

    # draw line between lower_point and laser
    points = [[0,max(xyz[:,1]),0], lower_point]
    #points = [[0.085,max(xyz[:,1]),0],[0.17,max(xyz[:,1]),0], model_robust.predict([max(xyz[:,1])][0],axis=1)]
    lines = [[0, 1]]
    colors = [[0, 1, 1] for i in range(len(lines))]
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)

    # find wire end
    #wire_end=find_point_on_the_line(lower_point,[0,max(xyz[:,1]),0],wire_thickness/2)
    wire_end=lower_point
    wire_end[2]-=wire_thickness
    
    wire_end_pcd = o3d.geometry.PointCloud()
    wire_end_pcd.points=o3d.utility.Vector3dVector([wire_end])
    wire_end_pcd.paint_uniform_color([0.1, 0.5, 0.5])
    #o3d.visualization.draw_geometries([xyz_pcd,xyz_pcd_inliers,lower_point_pcd,wire_end_pcd, line_set])

    return wire_end

In [13]:
def pick_element(element):
    anfahrpunkt=element.griffpunkt.copy()
    anfahrpunkt[2]+=0.04336
    ur.sendMoveL(zwischen_punkt,TCP, 30 / 500, 10,0)
    ur.waitForMessage()
    ur.OnRobot_width(40,10, False)
    ur.waitForMessage()
    ur.sendMoveL(anfahrpunkt,TCP, 30 / 500, 10,0)
    ur.waitForMessage()
    ur.sendMoveL(anfahrpunkt,TCP, 30 / 500, 10,0) # second time due bug. First one will be passed
    ur.waitForMessage()



    ur.sendMoveL(element.griffpunkt,TCP, 30 / 500, 10,0)
    ur.waitForMessage()
    ur.OnRobot_width(0,10, False)
    ur.waitForMessage()
    ur.sendMoveL(anfahrpunkt,TCP, 30 / 500, 10,0)
    ur.waitForMessage()
    ur.sendMoveL(anfahrpunkt,TCP, 30 / 500, 10,0)
    ur.waitForMessage()
    ur.sendMoveL(zwischen_punkt,TCP, 30 / 500, 10,0)
    ur.waitForMessage()
    ur.sendMoveL(element.scanpunkt,TCP, 30 / 500, 10,0)
    ur.waitForMessage()

In [14]:
def place_element(ur,p1,p2,transformation_matrix,socket,TCP,correction,element):
    
    wire_1_mat,wire_2_mat,grayimage=get_images(socket)
    wire_1_3d_cam= find_wire_end(wire_1_mat,element.drahtdicke)
    wire_2_3d_cam= find_wire_end(wire_2_mat,element.drahtdicke)
    print("wire_1_3d_cam: ",wire_1_3d_cam,"\n")
    print("wire_2_3d_cam: ",wire_2_3d_cam,"\n")

    wire_1_rob_coord=transform_to_base_coordinate(transformation_matrix,wire_1_3d_cam)
    wire_2_rob_coord=transform_to_base_coordinate(transformation_matrix,wire_2_3d_cam)
    
    if wire_1_rob_coord[0]>wire_2_rob_coord[0]:
        temp_wire=wire_1_rob_coord.copy()
        wire_1_rob_coord=wire_2_rob_coord.copy()
        wire_2_rob_coord=temp_wire.copy()
    print(wire_1_rob_coord)
    print(wire_2_rob_coord)

    print("p1: ",p1,"\np2: ",p2,"\nwire_1: ",wire_1_rob_coord,"\nwire_2: ",wire_2_rob_coord,"\nTCP: ",TCP,"\n")
    ur.formatSharedData()    
    current_point=[ur.ActualTcpPos[0],ur.ActualTcpPos[1],ur.ActualTcpPos[2]]
    axis_point=current_point.copy()
    axis_point[0]+=0.2
    
    print("wire_1_rob_coord[:3],current_point",wire_1_rob_coord[:3],current_point)
    v1=wire_1_rob_coord[:3]-current_point
    v2=wire_2_rob_coord[:3]-current_point
    print("v1: ",v1)
    print("v2: ",v2)
    

    PI=3.14159265
    tan_1 = (p2[1] - p1[1]) / (p2[0] - p1[0])  
    angle_1 = np.arctan(tan_1) 

    print("\n\n\np1p2: ",p1,p2)

    tan_2 = (wire_2_rob_coord[1] - wire_1_rob_coord[1]) / (wire_2_rob_coord[0] - wire_1_rob_coord[0])
    angle_2 = np.arctan(tan_2) 

    print(np.degrees(angle_1),np.degrees(angle_2))
    angle=angle_1 - angle_2
    
    #angle correction
    if(p2[0]<p1[0] and p2[1]<p1[1]):
        angle=angle-np.radians(180)
    
    v1_x=np.cos(angle) * (v1[0]) - np.sin(angle) * (v1[1]) 
    v1_y=np.sin(angle) * (v1[0]) + np.cos(angle) * (v1[1]) 
    v1_z=v1[2]
    
    v2_x=np.cos(angle) * (v2[0]) - np.sin(angle) * (v2[1]) 
    v2_y=np.sin(angle) * (v2[0]) + np.cos(angle) * (v2[1]) 
    v2_z=v2[2]
    
    v1_new = np.asarray([v1_x,v1_y,v1_z])
    v2_new = np.asarray([v2_x,v2_y,v2_z])
    
    print("v1_new",v1_new)
    print("v2_new",v2_new)
    print("current_point",current_point)
    
    
    

    path_v1=p1[:3]-(current_point+v1_new)
    path_v2=p2[:3]-(current_point+v2_new)
    path_v1[0]+=correction[0]
    path_v1[1]+=correction[1]
    hight=0.0135-wire_1_rob_coord[2]
    print("path_v1,np.degrees(angle)",path_v1,np.degrees(angle))
    
    temp_rotation=element.scanpunkt[3:].copy()
    temp_rotation[2]+=angle
    print("error_1: ",p1[0]-(current_point[0]+path_v1[0]+v1_new[0]),p1[1]-(current_point[1]+path_v1[1]+v1_new[1]))
    print("error_2: ",p2[0]-(current_point[0]+path_v1[0]+v2_new[0]),p2[1]-(current_point[1]+path_v1[1]+v2_new[1]))    
    
    ur.formatSharedData()
    start_point=element.scanpunkt.copy() #[-0.445, -0.28, 0.29423, -3.141, -0.008, 3.14]

    start_point[2]+=0.05
    print("start_point",start_point)
    ur.sendMoveL(start_point,TCP, 30 / 500, 10,0)
    ur.waitForMessage()


    ur.sendMoveL([board_anfahr[0],board_anfahr[1],board_anfahr[2],board_anfahr[3],board_anfahr[4],
                  board_anfahr[5]
                  ],TCP, 30 / 500, 10,0)
    ur.waitForMessage()

    ur.sendMoveL([current_point[0]+path_v1[0],current_point[1]+path_v1[1],current_point[2]+hight+0.05,
          temp_rotation[0],temp_rotation[1],temp_rotation[2]],TCP, 30 / 500, 10,0)
    ur.waitForMessage()    
    
    #this piece is used for estimation of correction values. Switch robot to local mode,
    #move TCP in -Z direction until you almost touch the board, and then get correction value.
    #line=None
    #while (line!="ready"):
    #    line=input(Fore.GREEN+"Input 'ready' to proceed:\n"+Style.RESET_ALL)

    ur.sendMoveL([current_point[0]+path_v1[0],current_point[1]+path_v1[1],current_point[2]+hight,
          temp_rotation[0],temp_rotation[1],temp_rotation[2]],TCP, 30 / 1000, 10,0)
    ur.waitForMessage()       
    ur.OnRobot_width(30,10, False)
    ur.waitForMessage()
    

    ur.sendMoveL([current_point[0]+path_v1[0],current_point[1]+path_v1[1],current_point[2]+hight+0.05,
          temp_rotation[0],temp_rotation[1],temp_rotation[2]],TCP, 30 / 1000, 10,0)
    ur.waitForMessage() 

    ur.sendMoveL([current_point[0]+path_v1[0],current_point[1]+path_v1[1],current_point[2]+hight+0.05,
          temp_rotation[0],temp_rotation[1],temp_rotation[2]],TCP, 30 / 1000, 10,0)
    ur.waitForMessage()    

    


# Get actual robot data



In [15]:
### Start UR communication
ur = UrCommunication.UrRobot('Parameters.json')
ur.connectUr()
ur.startUpdateProcess("data")
ur.formatSharedData()


### can be used to estimate of reference points.

In [16]:
# this points are mesured direct after TCP calibration while needle is still in the gripper
reference_point_1=[-0.21446885468013296,-0.42969758245980794, 0.08]
reference_point_2=[-0.21446885468013296,-0.42969758245980794, 0]
reference_point_3=[-0.15349175487064978, -0.4371083701823132, 0]
reference_points=[reference_point_1,reference_point_2,reference_point_3]

In [17]:
### Load camera-base transformation matrix
transformation_matrix=load_transformation_matrix()

[[ 0.99955423  0.00579947  0.02928669 -0.45443356]
 [-0.02910513 -0.029243    0.99914851 -0.49492601]
 [ 0.00665097 -0.99955551 -0.02906117  0.00452467]
 [ 0.          0.          0.          1.        ]]
rotations:  -91.66535362589488 -0.3810750968188738 -1.66787379606247


### Load place positions

In [18]:
place_positions=load_and_transform_place_positions(reference_points)


board_coordinate_system_points
 [(0.03415, 0.030780000000000002, 0.08), (0.03415, 0.030780000000000002, 0), (0.04177, 0.09174, 0)]

reference_points
 [[-0.21446885468013296, -0.42969758245980794, 0.08], [-0.21446885468013296, -0.42969758245980794, 0], [-0.15349175487064978, -0.4371083701823132, 0]]
matricies:
[[0.03415 0.03415 0.04177]
 [0.03078 0.03078 0.09174]
 [0.08    0.      0.     ]] [[-0.21446885 -0.21446885 -0.15349175]
 [-0.42969758 -0.42969758 -0.43710837]
 [ 0.08        0.          0.        ]]
det(R) < R, reflection detected!, correcting for it ...

R [[ 3.41416389e-03  9.99994171e-01  3.36588069e-05]
 [-9.99994172e-01  3.41416403e-03 -4.09068771e-06]
 [-4.20558055e-06 -3.36446444e-05  9.99999999e-01]]
t [[-2.45369020e-01]
 [-3.95652414e-01]
 [ 1.87355941e-06]]
Transformation matrix created.
[[ 3.41416389e-03  9.99994171e-01  3.36588069e-05 -2.45369020e-01]
 [-9.99994172e-01  3.41416403e-03 -4.09068771e-06 -3.95652414e-01]
 [-4.20558055e-06 -3.36446444e-05  9.99999999e-01 

In [19]:
### Move to start position and initialize tcp variables
point=scanpunkt.copy()
point[2]+=0.15

ur.sendMoveL(point,
             TCP, 30 / 1000, 10,0)
ur.waitForMessage()

ur.sendMoveL(scanpunkt,
             TCP, 30 / 1000, 10,0)
ur.waitForMessage()

b'Ready'
b'Ready'


b'Ready'

In [20]:
### connect to zmq and take pictures
context = zmq.Context()
socket = context.socket(zmq.REQ)
port = "5556"
socket.connect ("tcp://localhost:%s" % port)

In [21]:
correction=[-0.0004,-0.0002]# correction according robot axes x and y respectively
for i,element in enumerate(tht_elemente):
    place_position=place_positions[i]
    pick_element(element)
    place_element(ur,place_position[0],place_position[1],transformation_matrix,socket,TCP,correction,element)

b'Ready'
b'Ready'
b''
b'Ready'
b'Ready'
b'Ready'
b''
b'Ready'
b'Ready'
b'Ready'
Received reply from server: b'10'
wire_1.exr loaded. Shape:  (1544, 2064, 3)
wire_2.exr loaded. Shape:  (1544, 2064, 3)
663
model.params (array([ 0.0028579 , -0.01684857,  0.21645261], dtype=float32), array([ 0.00931824, -0.99957967, -0.02745378], dtype=float32))
vector [ 0.00931824 -0.99957967 -0.02745378]
541
model.params (array([-0.00464241, -0.01716075,  0.21466829], dtype=float32), array([ 0.02453622,  0.9995818 , -0.01530377], dtype=float32))
vector [ 0.02453622  0.9995818  -0.01530377]
wire_1_3d_cam:  [ 0.00283922 -0.01484424  0.21580766] 

wire_2_3d_cam:  [-0.00459299 -0.01514719  0.21393746] 

[-0.45284682 -0.28059409  0.01341731  1.        ]
[-0.4453614  -0.27895066  0.01310958  1.        ]
p1:  [-2.14472606e-01 -4.29697127e-01  6.94356678e-07  1.00000000e+00] 
p2:  [-2.14446590e-01 -4.37317082e-01  6.62310154e-07  1.00000000e+00] 
wire_1:  [-0.45284682 -0.28059409  0.01341731  1.        ] 
wire_2

model.params (array([ 0.00782363, -0.02448555,  0.21165606], dtype=float32), array([ 0.08171258,  0.99656004, -0.01382673], dtype=float32))
vector [ 0.08171258  0.99656004 -0.01382673]
wire_1_3d_cam:  [ 0.00024862 -0.02112334  0.21068491] 

wire_2_3d_cam:  [ 0.00804845 -0.02174372  0.21091802] 

[-0.44813729 -0.28381003  0.01951753  1.        ]
[-0.44033771 -0.28378598  0.02018273  1.        ]
p1:  [-1.93407288e-01 -4.69295437e-01 -1.76661110e-07  1.00000000e+00] 
p2:  [-1.87948747e-01 -4.74716832e-01 -3.82566334e-07  1.00000000e+00] 
wire_1:  [-0.44813729 -0.28381003  0.01951753  1.        ] 
wire_2:  [-0.44033771 -0.28378598  0.02018273  1.        ] 
TCP:  [0.0, 0.0, 0.0] 

wire_1_rob_coord[:3],current_point [-0.44813729 -0.28381003  0.01951753] [-0.44500116193899925, -0.27999685712437394, 0.24423472636653723]
v1:  [-0.00313612 -0.00381317 -0.2247172 ]
v2:  [ 0.00466345 -0.00378913 -0.224052  ]



p1p2:  [-1.93407288e-01 -4.69295437e-01 -1.76661110e-07  1.00000000e+00] [-1.87948747e-

b'Ready'
b'Ready'
b'Ready'
b'Ready'
b'Ready'
b''
b'Ready'
